**Generate random string**


Create random string provided a len/ allow case sensitive or not  

In [1]:
DECLARE @seed INT; 
DECLARE @allow_number BIT; 
DECLARE @upper_only BIT; 
DECLARE @real_random INT; 
DECLARE @len SMALLINT; -- len of the string 
DECLARE @value VARCHAR(20); --a max of 20 in length. If you need more, just increase this
DECLARE @run SMALLINT; 

SET @len=20 
--clear string 
SET @value='' 
SET @upper_only=0 
SET @allow_number=1

WHILE ( len(@value) < @len ) -- not yet there 
  BEGIN 
      SET @seed=( Abs(Cast(Cast(Newid() AS VARBINARY) AS INT)) ) 
      SET @real_random=Cast(Round(Rand(@seed), 10) * 1000000000 AS INT) 
      -- take a character or a number 
      IF( @allow_number = 1 AND @real_random % 35 > 25 ) -- a number 
        SET @value += Cast(( @real_random%35 ) - 26 AS VARCHAR(1)) 
      ELSE 
        BEGIN 
            IF ( @real_random%2 = 1 
                 AND @upper_only = 0 ) 
              SET @value +=Lower(Char(65 + ( @real_random % 25 ))) 
            ELSE 
              SET @value += Char(65 + ( @real_random % 25 )) 
        END 
      --print ('Value at run'+cast(@run as varchar(2))+': '+@value) 

  END 

PRINT @value 

UNwidN4cCt78F20Ug3Kv

Total execution time: 00:00:00.440

But we could not create a function with rand() or newid() in sql server. Should go around with the solution

1. Create a view hold the random number
2. Create a function and capture values from that view

In [3]:
CREATE VIEW random_vals
AS
Select random_val
from (values 
			(Cast(Round(Rand(( Abs(Cast(Cast(CRYPT_GEN_RANDOM(2) AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
			(Cast(Round(Rand(( Abs(Cast(Cast(Newid() AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
            (Cast(Round(Rand(( Abs(Cast(Cast(CRYPT_GEN_RANDOM(2) AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
			(Cast(Round(Rand(( Abs(Cast(Cast(Newid() AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
            (Cast(Round(Rand(( Abs(Cast(Cast(CRYPT_GEN_RANDOM(2) AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
			(Cast(Round(Rand(( Abs(Cast(Cast(Newid() AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
            (Cast(Round(Rand(( Abs(Cast(Cast(CRYPT_GEN_RANDOM(2) AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
			(Cast(Round(Rand(( Abs(Cast(Cast(Newid() AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
            (Cast(Round(Rand(( Abs(Cast(Cast(CRYPT_GEN_RANDOM(2) AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
			(Cast(Round(Rand(( Abs(Cast(Cast(Newid() AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
            (Cast(Round(Rand(( Abs(Cast(Cast(CRYPT_GEN_RANDOM(2) AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
			(Cast(Round(Rand(( Abs(Cast(Cast(Newid() AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
            (Cast(Round(Rand(( Abs(Cast(Cast(CRYPT_GEN_RANDOM(2) AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
			(Cast(Round(Rand(( Abs(Cast(Cast(Newid() AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
            (Cast(Round(Rand(( Abs(Cast(Cast(CRYPT_GEN_RANDOM(2) AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
			(Cast(Round(Rand(( Abs(Cast(Cast(Newid() AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
            (Cast(Round(Rand(( Abs(Cast(Cast(CRYPT_GEN_RANDOM(2) AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
			(Cast(Round(Rand(( Abs(Cast(Cast(Newid() AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
            (Cast(Round(Rand(( Abs(Cast(Cast(CRYPT_GEN_RANDOM(2) AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT)),
			(Cast(Round(Rand(( Abs(Cast(Cast(Newid() AS VARBINARY) AS INT)) )), 10) * 1000000000 AS INT))
	 ) as random_values(random_val) 


Commands completed successfully.

Total execution time: 00:00:00.265

In [ ]:
ALTER FUNCTION generate_random_string(@len SMALLINT,@allow_number BIT=1, @upper_only BIT=1)
RETURNS VARCHAR(20)
AS
BEGIN
--select top (len) row from the view, get the number and character from it,
--string_agg will be the better choice but it works only with @@version>= 2017 :-( 
DECLARE @value VARCHAR(20); --a max of 20 in length. If you need more, just increase this
SELECT @value=Substring(
	(SELECT random_char+''
	 FROM (
		SELECT  random_val, 
				--if we allow number and the value is a number, go get it
				CASE WHEN @allow_number=1 and random_val%35 >25 THEN cast((random_val%35)-26 AS VARCHAR(1))
				--if we allow all case, and a random vals said we should generate a lower case value, do it
				WHEN @upper_only=0 AND random_val%2 =1 THEN LOWER(CHAR(65 + (random_val % 25 ))) 
				-- just keep it upper case
				ELSE CHAR(65 +(random_val% 25 ))
				END AS random_char
		FROM (
				SELECT ROW_NUMBER() OVER (ORDER BY random_val) rand_order, random_val 
				FROM random_vals
			 ) AS random_values
		WHERE rand_order < @len+2
) AS val
FOR XML PATH('')),2,@len);
RETURN @value
END



In [46]:
DECLARE @allow_number BIT; 
DECLARE @upper_only BIT; 
DECLARE @len SMALLINT; -- len of the string 
DECLARE @value VARCHAR(20); --a max of 20 in length. If you need more, just increase this


SET @len=5
--clear string 
SET @value='' 
SET @upper_only=1
SET @allow_number=0

PRINT 'Random string len '+cast(@len as varchar(2))+':    '+ DBO.generate_random_string(@LEN,@allow_number,@upper_only)

Random string len 5: YNITP

Total execution time: 00:00:00.228